In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0-dev20200315


In [4]:
# Implementation of AlexNet in Functional API mode
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers, datasets, utils
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Flatten, Dropout

#tf.keras.backend.clear_session()
 
(x, y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis=-1)
y_test = tf.squeeze(y_test, axis=-1)

def preprocess(x, y):
    x = 2 * tf.cast(x, dtype = tf.float32)/255. -1
    y = tf.cast(y, dtype = tf.int32)
    return x, y

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(128)

alex_net_inputs = Input(shape=[32, 32, 3])
x = Conv2D(filters = 96, kernel_size=[11,11], strides = 4, padding = [[0,0],[2,2], [2,2],[0,0]], activation='relu')(alex_net_inputs)
x = MaxPool2D(pool_size=[3,3], strides = 2, padding='valid')(x)

x = Conv2D(filters = 256, kernel_size = [5,5], strides = 1, padding = [[0,0],[2,2], [2,2],[0,0]], activation = 'relu')(x)
x = MaxPool2D(pool_size=[3,3], strides = 2, padding='valid')(x)

x = Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1], [1,1],[0,0]], activation = 'relu')(x)
x = Conv2D(filters = 384, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1], [1,1],[0,0]], activation = 'relu')(x)
x = Conv2D(filters = 256, kernel_size = [3,3], strides = 1, padding = [[0,0],[1,1], [1,1],[0,0]], activation = 'relu')(x)

x = MaxPool2D(pool_size=[3,3], strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)

x = Dense(4096, activation='relu')(x)
x = Dropout(rate = 0.2)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate = 0.2)(x)
alex_net_outputs = Dense(10, activation=None)(x)



model = Model(inputs = alex_net_inputs, outputs = alex_net_outputs, name ='alexnet')
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #optimizer=keras.optimizers.RMSprop(),
              optimizer=keras.optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

history = model.fit(train_db, 
                    batch_size=64,
                    epochs=100)

test_scores = model.evaluate(test_db, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])
#
#utils.plot_model(model_alexnet, 'test.png')


Epoch 1/100
391/391 [==============================] - 5s 14ms/step - loss: 1.5453 - accuracy: 0.4242
Epoch 2/100
391/391 [==============================] - 5s 14ms/step - loss: 1.2431 - accuracy: 0.5503
Epoch 3/100
391/391 [==============================] - 5s 13ms/step - loss: 1.1137 - accuracy: 0.6007
Epoch 4/100
391/391 [==============================] - 5s 13ms/step - loss: 1.0153 - accuracy: 0.6364
Epoch 5/100
391/391 [==============================] - 5s 13ms/step - loss: 0.9413 - accuracy: 0.6662
Epoch 6/100
391/391 [==============================] - 6s 14ms/step - loss: 0.8696 - accuracy: 0.6943
Epoch 7/100
391/391 [==============================] - 5s 13ms/step - loss: 0.8098 - accuracy: 0.7151
Epoch 8/100
391/391 [==============================] - 5s 14ms/step - loss: 0.7455 - accuracy: 0.7368
Epoch 9/100
391/391 [==============================] - 5s 13ms/step - loss: 0.6881 - accuracy: 0.7587
Epoch 10/100
391/391 [==============================] - 5s 13ms/step - loss: 0.637

In [11]:
# Implementation of VGG16 in Functional API mode
from numpy import random
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, BatchNormalization, Flatten, Dropout
from tensorflow.keras import datasets, optimizers, losses, utils, Model, Input

(x, y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis = -1)
y_test = tf.squeeze(y_test, axis = -1)

x = 2 * x.reshape(x.shape[0],32,32,3).astype('float32') / 255 - 1
#random.shuffle(x)
x_test = 2* x_test.reshape(x_test.shape[0],32,32,3).astype('float32')/ 255 -1

train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.batch(128)

vgg16_inputs = Input(shape=[32,32,3])
x = Conv2D(filters = 64, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(vgg16_inputs)
x = Conv2D(filters = 64, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 128, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 128, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 256, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = Conv2D(filters = 512, kernel_size=[3,3], padding='same',strides = 1, activation=tf.nn.relu)(x)
x = MaxPool2D(pool_size=[2,2], strides=2, padding='valid')(x)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(rate=0.5)(x)

vgg16_outputs = Dense(10, activation=None)(x)

model_vgg16 = Model(vgg16_inputs, vgg16_outputs, name = 'VGG16')
model_vgg16.summary()

# CategoricalCrossentropy for one-hot encoded y, e.g. [0,1,0]
# SparseCategoricalCrossentropy for integer labels, e.g. [2, 3]
model_vgg16.compile(optimizer = optimizers.Adam(lr=1e-4), 
              loss = losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics = ['accuracy'])

training_log = model_vgg16.fit(train_db, batch_size = 128, epochs = 100)

testing_results = model_vgg16.evaluate(test_db, verbose = 2)
print('Test loss: ', testing_results[0])
print('Test Accuracy: ', testing_results[1])

Model: "VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_121 (Conv2D)          (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 8, 8, 128)         0     

391/391 [==============================] - 12s 31ms/step - loss: 0.0316 - accuracy: 0.9896
Epoch 45/100
391/391 [==============================] - 12s 30ms/step - loss: 0.0329 - accuracy: 0.9900
Epoch 46/100
391/391 [==============================] - 12s 30ms/step - loss: 0.0329 - accuracy: 0.9900
Epoch 47/100
391/391 [==============================] - 12s 31ms/step - loss: 0.0289 - accuracy: 0.9908
Epoch 48/100
391/391 [==============================] - 12s 30ms/step - loss: 0.0343 - accuracy: 0.9895
Epoch 49/100
391/391 [==============================] - 12s 30ms/step - loss: 0.0294 - accuracy: 0.9906
Epoch 50/100
391/391 [==============================] - 12s 30ms/step - loss: 0.0290 - accuracy: 0.9906
Epoch 51/100
391/391 [==============================] - 12s 30ms/step - loss: 0.0301 - accuracy: 0.9909
Epoch 52/100
391/391 [==============================] - 12s 30ms/step - loss: 0.0270 - accuracy: 0.9919
Epoch 53/100
391/391 [==============================] - 12s 30ms/step - loss: